## Original data CNN learning curve

## Conventional transformation image samples

## Conventional transformation optimisation
### Accuracy vs intensity gamma

### Accuracy vs ratio alpha

### Learning curve on full data with augmentation

## ImageGPT output samples

## GAN output samples

## Accuracy vs FID
### Conventional SD

### ImageGPT

### ??GANs??

## Accuracy vs alpha
### Conventional vs ImageGPT

### Conventional vs GAN

### ??All together with real-only??